# Overview

The purpose of this project is to build an optimal machine learning model that could identify credit card fraud transactions through comparison of different statistical methods and their hyper-parameters tuning. 

# Data Introduction

The dataset could be accessed [here](https://www.kaggle.com/mlg-ulb/creditcardfraud). This dataset contains credit card transactions from a European bank in September 2013. There are 492 fraud transactions out of 284,807 transactions. Due to confidentiality protection, the feature variables only show from V1 to V28, transaction time, amount of transaction, and class which states 0 (normal) and 1 (fraud). V1 - V28 are components of PCA transformation on original features. 

After checking data types and null values, I found out that all features are float variables, and there is no null values. By looking into the value counts for each class and checking with data description, we can see that the dataset is extremely imbalanced. The majority of class is 0, which are normal transactions, while only a small portion of the dataset are fraud transactions. Due to the extreme imbalance, a correlation analysis among variables would not be as effective as expected.
![image](./img/EDA_class_count.png)

# Rebalance data with SMOTE

The SMOTE method allows me to create a dataset with more balanced Classes. In SMOTE train dataset, there are 213,226 Class 1 and 213,226 Class 0. I trained my machine learning models with SMOTE dataset and validated the performances with train (70%) and test (30%) of the original dataset.

![image](./img/EDA_smote_class.png)

# Model Default Parameters Performance

The machine learning models I tried include Random Forest, Extratrees, Xgboost, Gradient Boost, Adaboost, Logistic Regression, and SVM. I used Recall, Precision, False Negative, False Positive, and Roc-Auc scores as evaluation metrics for model performance. In this project, False Negative represents the cases that the model predicts a transaction as normal while it is actually fraud, and False Positive represents the cases that the model predicts a transaction as fraud while it is actually normal. From a bank's perspective, I think a false negative could cause more damage than a false positive, so maximizing recall scores while maintaining a high precision is crucial for the model training.

![image](./img/baseline.png)

From the table above, we could see that tree methods such as Random Forest and Extratrees perform relatively balanced based on their high recall and precision scores. Boosting methods such as Xgboost and Gradient Boost have higher recall scores, but their precision scores need significant improvements. All models seem to overfit with default parameters by comparing their train and test scores.

# Parameter Tuning

## Gradient Boost

### default parameter

There are a number of parameters in Gradient Boost Classifier that we could tune such as learning_rate, n_estimators, subsample, etc.

The first Gradient Boost is trained with its default parameters, with learning_rate = 0.1, n_estimator = 100, subsample = 1, min_samples_split = 2, min_samples_leaf = 1, max_depth = 3.

![image](./img/gradboost/default_table.png)

From the table above, we could see that the default model performs well in terms of recall and false negative. The model has 10 false negatives on test data, which means that 10 fraud transactions are predicted as normal transactions. However, the default model does not perform really well with a low precision score and high false positive. It has 920 false positives on test data, so 920 normal transactions are classified as fraud transactions. Both train and test data show a similar traits of high false positive and low precision scores.

The following confusion matrixes could better visualize the model performance in terms of True Positive, False Positive, True Negative, and False Negative in both train and test data.


![image](./img/gradboost/default_train.png)
![image](./img/gradboost/default_test.png)

By looking at the feature importance in the default model, we could see that predictors V14 significantly dominated the importance in this model as it weighs almost 80% in the model, while the other features' effects are minimized.

![image](./img/gradboost/default_feature.png)

## Gridsearch to find  optimal parameters

I decided to focus on the following parameters to improve the model performance:
- min_samples_leaf & min_samples_split
- max_depth
- subsample
- n_estimators & learning_rate

In each step of Gridsearch, I focused on 1~2 related parameters and fit with 5 times cross-validation to minimize the chance of randomness. 

### Step 1: min_samples_leaf & min_samples_split

I first tried to tune the model's min_samples_leaf and min_sample_split. These 2 parameters control the condition for a tree node to split. Tuning these 2 parameters could help resolve overfit. The default for min_samples_split and min_samples_leaf are 2 and 1. After gridsearch on different input, I found the best inputs to be min_samples_leaf = 10 and min_samples_split = 500.

![image](./img/gradboost/gs2_table.png)

The model performance only improved slightly. We could see a minimal decrease in number of false positives in test data compare to the default parameters, a slight increase in precision and roc-auc scores. Overall, these inputs did not bring significant improvement to the model.

However, we could see a great change in the model's feature importance. V14 no longer have the highest importance score in the model. There is a significant boost in scores for V11, V10, and V4 as their importance scores reach above 0.1. V11 has the highest score slightly above 0.3, which is more balanced than the default model with V14's 0.8. 

![image](./img/gradboost/gs2_feature.png)

### Step 2: max_depth & subsample

In step 2, I used Gridsearch to tune max_depth and subsample. Max_depth controls the depth of the tree model. Usually a larger max_depth would result in a more-detailed learning on train data. Subsample measures the percentage of train data used in each individual base learner. The default for max_depth is 3 and subsample is 1, which is the full dataset. After gridsearch, the best parameter inputs appear to be max_depth = 20 and subsample = 0.9.

![image](./img/gradboost/gs3_table.png)

We could see that the model performance improved significantly for precision and train data. Compare to the previous model performances with precision scores around 10%, the current model improves the train precision score to 100% and test precision score to 82.2%. We could also find a sharp decline in number of false positives in both train and test data. With 100% recall and precision score in train data, the model correctly classifies all transactions in the train dataset and achieves 0 false positive and false negative. The number of false positive in the test dataset is also reduced from previously 910 to 21. 

However, there is also a slightly increase in false negative and decrease in recall and roc-auc score for the test dataset. From the table above, we see an increase in false negative from 10 to 16, a 5% decrease in recall score and 3% decrease in roc-auc score for test data. But compare with the improvements made on precision and false positive, these inputs should still be considered effective.

The following confusion matrixes could better visualize the model performance in terms of True Positive, False Positive, True Negative, and False Negative in both train and test data.

![image](./img/gradboost/gs3_train.png)
![image](./img/gradboost/gs3_test.png)

Based on the feature importance chart, the feature importance scores seem to be more balanced as scores steadily increase for most of the features. V11's importance score shows a slight decrease to below 0.3

![image](./img/gradboost/gs3_feature.png)

### Step 3: n_estimators & learning_rate

In step 3, I applied Gridsearch on n_estimators and learning_rate. N_estimators decides the number of trees the model uses, and learning_rate controls the weighting of each tree in the model. In most cases, a higher number of n_estimators and lower learning_rate result in a more thorough learning of train data, and there exists a trade-off between learning_rate and n_estimators, so in practice, if we decide to multiply n_estimators by 2, we should also divide learning_rate by 2 at the same time. 

The default value for n_estimator is 100 and learning_rate is 0.1. In this step, I set the input list for n_estimators to be [50, 100, 300, 500] and learning_rate to be [0.05, 0.1, 0.3, 0.5]. Based on the results of Gridsearch, the best parameters input for n_estimator is 300 and learning_rate is 0.1

![image](./img/gradboost/gs5_table.png)

Compare to the step 2 model inputs, the current inputs improve the overall model performance only by a slight percentage. There is around 2% improvement in test recall score, 1% improvement in precision and roc-auc scores in test data. The model continues to perform perfectly in train data. The number of false positive decreases by 1 and false negative decreases by 2. The feature importance does not change much compare to step 2.

The following confusion matrixes could better visualize the model performance in terms of True Positive, False Positive, True Negative, and False Negative in both train and test data.

![image](./img/gradboost/gs5_train.png)
![image](./img/gradboost/gs5_test.png)


### Step 4: increase n_estimator and reduce learning_rate by 5

In step 4, I continued to tune n_estimators and learning_rate from step 3 by multiplying n_estimators and divide learning_rate by 5, so in this model, n_estimators = 1500 and learning_rate = 0.02. 

![image](./img/gradboost/gs8_table.png)

This model inputs doesn't seem to perform better than step 4 inputs. The following confusion matrixes could better visualize the model performance in terms of True Positive, False Positive, True Negative, and False Negative in both train and test data.

![image](./img/gradboost/gs8_train.png)
![image](./img/gradboost/gs8_test.png)

However, the feature importance scores are more balanced compared to the previous model inputs. V10 becomes the feature with highest importance scores, with a score around 0.175. The scores for the other features also have visible improvements based on the feature importance chart below.

![image](./img/gradboost/gs8_feature.png)

## Gradient Boost Summary

### Default model performance:

![image](./img/gradboost/default_table.png)

### Step 1 model performance:

![imgae](./img/gradboost/gs2_table.png)

### Step 2 model performance:

![image](./img/gradboost/gs3_table.png)

### Step 3 model performance:

![image](./img/gradboost/gs5_table.png)

### Step 4 model performance:

![image](./img/gradboost/gs8_table.png)

Tuning parameters could bring significant improvement in model performance. Model inputs at step 3 outputs the best model performance with the following parameter inputs:
- min_samples_leaf = 15
- min_samples_split = 500
- max_depth = 20
- subsample = 0.9
- n_estimator = 300
- learning_rate = 0.1

Compare to default model, the optimal model at step 3 improves train recall by 3%, train precision by 88%, train roc-auc by 2%, test precision by 73%. It also reduces the number of false positive in the test dataset by 97.8%. However, the number of false negative in the test dataset increases by 4, and test recall score decreases by 3.5%.